<a href="https://colab.research.google.com/github/marcsinner/bwi/blob/main/bwi_rag_multiple_pdfs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install langchain-community chromadb sentence-transformers langchain-groq rank_bm25

In [4]:
import os
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')


In [5]:
from langchain_community.document_loaders import WebBaseLoader


In [6]:
articles = ["https://www.presseportal.de/pm/67428/5842007", "https://www.presseportal.de/pm/67428/5835949", "https://www.presseportal.de/pm/67428/5829929",
            "https://www.presseportal.de/pm/67428/5828109", "https://www.presseportal.de/pm/67428/5823073", "https://www.presseportal.de/pm/67428/5821522",
            "https://www.presseportal.de/pm/67428/5819787", "https://www.presseportal.de/pm/67428/5819366", "https://www.presseportal.de/pm/67428/5819074"
            ]

In [7]:
data = []

for article in articles:
  loader = WebBaseLoader(article)
  data += loader.load()

In [8]:
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (SentenceTransformerEmbeddings)
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import CharacterTextSplitter


# split it into chunks TODO use spacy for sentences?
# TODO overlap
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(data)

# create the open-source embedding function
# TODO use better model?
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
# TODO try aari1995/German_Semantic_V3 from huggingface

# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)

<ipython-input-8-690d98522fb4>:14: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
docs[0] # todo preprocessing (cleaner functions, e.g. cleantext())

Document(metadata={'source': 'https://www.presseportal.de/pm/67428/5842007', 'title': 'Medikamentenlieferung in See: Deutscher Marineverband hilft italienischem ... | Presseportal', 'description': 'Presse- und Informationszentrum Marine - Rostock - Am heutigen Montag, den 12. August 2024, lieferte ...✚ Mehr lesen', 'language': 'de'}, page_content='Medikamentenlieferung in See: Deutscher Marineverband hilft italienischem ... | Presseportal\n\n\nPRESSEPORTAL\nPresseportal Logo\n\n\nPRESSEPORTAL\nPresseportal Logo\n\nStorysBlaulichtRegionalMeine AbosStorysBlaulichtRegionalMeine AbosRechercheAlle StorysBeliebte StorysNewsrooms A-ZAlle DienststellenInfoüber Presseportal.deStory/Meldung einstellenThemenAuto / VerkehrBau / ImmobilienFashion / BeautyFinanzenGesundheit / MedizinHandelMedien / KulturNetzweltPanoramaPeoplePolitikPresseschauSozialesSportTourismus / UrlaubUmweltWirtschaftWissen / Bildung\xa0mehr Themen Auto / VerkehrBau / ImmobilienFashion / BeautyFinanzenGesundheit / MedizinHandel

In [10]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq

# Define the repository ID for the Gemma 2b model
repo_id = "microsoft/Phi-3.5-mini-instruct"

# Set up a Hugging Face Endpoint for Gemma 2b model
#llm = HuggingFaceEndpoint(
#    repo_id=repo_id, max_length=1024, temperature=0.1
#)

llm = ChatGroq(
    model="llama-3.1-8b-instant",
)

In [11]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import RetrievalQA
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever

# todo confidence score for k results
chroma_retriever = db.as_retriever(search_type="mmr", search_kwargs={'k': 4, 'fetch_k': 20})

# todo lemmatize and stemming,
# todo confidence score for k results, add threshold (if no results, return 'dont know')
bm25_retriever = BM25Retriever.from_documents(
    data
)

prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, chroma_retriever], weights=[0.5, 0.5]
)

rag_chain = (
    {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:5431: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


In [12]:
result = rag_chain.invoke("Wer ist kapitän des BWI schiffs?")
print(result.content)
# TODO chat history
# TODO german sentence embedding
# TODO validation: 'reverse validation', sind die chunks sinnvoll zu den questions?
# TODO Fragen stellen die nicht beantwortet werden können
# bringt das BM25 was?
# TODO prio 1: evaluation via eigenem scoring? (e.g. teacher model, ), llm metrics : production RAG:  https://towardsdatascience.com/17-advanced-rag-techniques-to-turn-your-rag-app-prototype-into-a-production-ready-solution-5a048e36cdc8
# idea: automatic chunk science via metric (e.g. text size)
# TODO chunks anschaun mit guten und schlechten fragen
# mixture of experts

Das Segelschulschiff "Gorch Fock" wird von Fregattenkapitän Elmar Bornkessel kommandiert. Er übernahm das Kommando am 25. Juli 2024 von Kapitän zur See Andreas-Peter Graf von Kielmansegg.


In [32]:
history = ""
def ask_question(question):
  cur_history = "\n" + "User: "+question + "\n + AI:"
  result = rag_chain.invoke(history + cur_history)
  cur_history += result.content + "\n"
  return result.content, cur_history

In [34]:
result, cur_history = ask_question("Wie heist er?")
history += cur_history
print(result)

Der Kapitän ist Kapitän zur See Carsten Holtgreve. Er ist 54 Jahre alt.


In [35]:
history

'\nUser: Wie ist der Kapitän?\n + AI:Der Kapitän ist Kapitän zur See Carsten Holtgreve (54).\n\nUser: Wie heist er?\n + AI:Der Kapitän ist Kapitän zur See Carsten Holtgreve. Er ist 54 Jahre alt.\n'

## Evaluation using ragas and Weavate

In [39]:
#!pip install ragas

In [41]:
#pip install weave

Shitty package, pyarrow old version is used, not compatible with previous imports

## Own Offline Evaluation method

idea:
- create chunks of data
- for each chunk let ai generate a question-answer pair
- ask the RAG the questions
- evaluate answer using AI with yes or no output (0-1 scoring works bad with AI)
- output average score

In [115]:
import json

# TODO removed pipeline, as groq models gets overused, next: use locally run LLM
#def evaluation_pipeline(docs, rag_chain, llm):

# create JSON with Q&As
bundles = []
#for doc in docs:
doc = docs[6]
context = doc.page_content
response = llm.invoke("Erstell mir eine json im format {question: Frage, answer: Antwort}. Gibt nur die json zurück. Dabei erstellst du aus dem folgendem context eine frage und antwort. context: "+context)
# json data preperation:
json_qa = response.content.replace("```json\n", "").replace("\n```", "").replace("```", "").replace('\n', '')
json_qa = "{\"bundle\" : ["+json_qa+"]}"
json_qa= json.loads(json_qa)
bundles += json_qa['bundle']



In [122]:
yes_amount = 0
no_amount = 0
invalid_amount = 0
# evaluate Q&As
for qa in bundles:
  question = qa['question']
  answer = qa['answer']
  rag_answer = rag_chain.invoke(question)
  rag_answer = rag_answer.content
  response = llm.invoke('Gib nur Ja oder Nein zurück. Hier ist eine Frage und zwei Antwort. Stimmt Antwort 1 mit Antwort 2 überein? Frage 1: ' +question+', Antwort 1:'+answer+', Antwort 2: '+ rag_answer)
  if "ja" in response.content.lower():
    yes_amount += 1
  elif "nein" in response.content.lower():
    no_amount += 1
    print("Wrong answer: " + question + " correct answer: " +answer+ " RAG answer: "+ rag_answer)
  else:
    print(response.content)
    invalid_amount += 1

score = yes_amount / (yes_amount + no_amount + invalid_amount)
print("Yes: " + str(yes_amount) + " No: " + str(no_amount) + " Invalid: " + str(invalid_amount) )
print("Score: " + str(score))
#return score

# TODO add context to evulation query?

Wrong answer: Wann sind die Tore im Marinestützpunkt auf der Hohen Düne zum 'Open Stützpunkt' geöffnet? correct answer: Die Tore im Marinestützpunkt sind von Freitag bis Sonntag, den 09. bis 11. August 2024, geöffnet. RAG answer: Die Tore im Marinestützpunkt auf der Hohen Düne werden vom Freitag, den 09. August 2024, 10:00 Uhr - 18:00 Uhr bis Sonntag, den 11. August 2024, 10:00 Uhr - 16:00 Uhr geöffnet.
Wrong answer: Welche Flugzeuge werden bei der Hanse Sail 2024 gezeigt? correct answer: Ein 'Sea King' und ein 'Sea Lynx' werden bei der Hanse Sail 2024 gezeigt. RAG answer: Die Flugzeuge, die bei der Hanse Sail 2024 gezeigt werden, sind nicht spezifiziert. Es wird jedoch erwähnt, dass die Marineflieger ihr Können mit Hubschraubern der Deutschen Marine, inklusive einer simulierten Seenotrettung im Hafenbecken des Stützpunkts, präsentieren werden.
Yes: 4 No: 2 Invalid: 0
Score: 0.6666666666666666


Example output:
Yes: 5 No: 1 Invalid: 0
Score: 0.8333333333333334

Wrong answer: Was gibt es bei der 'Blaulichtfraktion' bei der Hanse Sail 2024 zu sehen? correct answer: Die 'Blaulichtfraktion' (THW, Bundes- und Wasserschutzpolizei sowie Stützpunktfeuerwehr) wird bei der Hanse Sail 2024 zu sehen sein. RAG answer: Bei der "Blaulichtfraktion" bei der Hanse Sail 2024 können die Besucher Teile des Minenjagdverbandes Ost- und Nordsee, das Heer und die Luftwaffe mit ihren Waffensystemen sowie das Helmtaucherteam sehen.


In [123]:
#score = evaluation_pipeline(docs, rag_chain, llm)

In [121]:
doc

Document(metadata={'source': 'https://www.presseportal.de/pm/67428/5835949', 'title': 'Willkommen an Bord - die Deutsche Marine bei der Hanse Sail 2024 | Presseportal', 'description': 'Presse- und Informationszentrum Marine - Warnemünde - Warnemünde - Von Freitag bis Sonntag, den 09. ...✚ Mehr lesen', 'language': 'de'}, page_content='3 Dokumente20240805_Hanse_Sail_PM_046_24.pdfPDF - 102 kBAkkreditierungWMD.pdfPDF - 276 kBProgrammHanseSail24.pdfPDF - 87 kB\nWarnemünde (ots) Warnemünde - Von Freitag bis Sonntag, den 09. bis 11. August 2024, werden die Tore im Marinestützpunkt auf der Hohen Düne zum "Open Stützpunkt" wieder geöffnet.Auch dieses Jahr freuen sich die Soldaten und Soldatinnen von Heer, Luftwaffe und Marine auf zahlreiche Gäste im Marinestützpunkt und am Passagierkai in Warnemünde."Mein Team und ich sind stolz auch in diesem Jahr wieder einer der Publikumsmagnete der maritimen Großveranstaltung zu sein. Insbesondere der direkte Austausch zwischen den Soldatinnen und Soldaten 